#Loading important Libraries

In [ ]:
# The essential Libaries for webscraping
from bs4 import BeautifulSoup
import requests
import pandas as pd

# To add a threshold to pause the code after 10 pages scraped
import time

#Scaping each job details (Additional infos)

In [ ]:
url_2 = 'https://www.jobberman.com/listings/tiktok-poster-7j86rg?_gl=1*wcv88i*_up*MQ..*_gs*MQ..&gclid=CjwKCAiA3fnJBhAgEiwAyqmY5bjRXQJqCH1h--u8jTguvWn1E8_GbKAW9FjKY__DimPqp4AojSNQXxoCxG4QAvD_BwE&gbraid=0AAAAADmiEX1l_xG5KDvNbdGx_6oOLvw1L'

def scrapeJobDetails(url):
  response = requests.get(url)
  src = response.content
  soup = BeautifulSoup(src, 'html.parser')

# Industries
  industry_element = soup.find_all('a', attrs={'class': 'inline-flex items-center gap-x-2 text-base leading-6 font-normal text-gray-900 hover:bg-gray-100'})
  industry_value = industry_element[2].text.replace('\n', '').strip() if industry_element else "N/A"

# Min_Qualification, ExperienceLevel, ExperienceLength
  details_spans = soup.find_all('span', attrs={'class': 'text-sm leading-5 font-normal text-gray-800 md:text-base md:leading-6'})

  Min_Qualification_value = details_spans[0].text.replace('\n', '').strip() if len(details_spans) > 0 else "N/A"
  ExperienceLevel_value = details_spans[1].text.replace('\n', '').strip() if len(details_spans) > 1 else "N/A"
  ExperienceLength_value = details_spans[2].text.replace('\n', '').strip() if len(details_spans) > 2 else "N/A"

  return industry_value, Min_Qualification_value, ExperienceLevel_value, ExperienceLength_value

#   Scraping Job pages

In [ ]:
def scrap_jobs(url ="https://www.jobberman.com/jobs?campaignid=21043129105&gad_campaignid=21039808007&gad_source=1&gbraid=0AAAAADmiEX1l_xG5KDvNbdGx_6oOLvw1L&gclid=CjwKCAiA3fnJBhAgEiwAyqmY5bjRXQJqCH1h--u8jTguvWn1E8_GbKAW9FjKY__DimPqp4AojSNQXxoCxG4QAvD_BwE"):
    #Getting request to scrap from a given website
    response = requests.get(url)
    #Loading the HTML document
    src = response.content

    #Parsing the document using Beautiful Soup
    soup = BeautifulSoup(src, 'html.parser')

    #Job Info lists
    job_title_list = []
    company_list = []
    job_function_list = []
    TimeList = []
    location_list = []
    Salary_list = []
    job_urls_final_list = [] # represent the final list of URLs
    workType_list = []
    industry_list = []
    Min_Qualification_list = []
    ExperienceLevel_list = []
    ExperienceLength_list = []

    companyNameTag = soup.find_all('p', attrs={'class': 'text-sm text-blue-700 text-loading-animate inline-block mt-3'}) # company name Tag
    job_title = soup.find_all('p', attrs={'class': 'text-lg font-medium break-words text-link-500'})  # job title Tag
    jobFunctionTag = soup.find_all('p', attrs={'class': 'text-sm text-gray-500 text-loading-animate inline-block'}) #Job Function Tag
    TimeTag = soup.find_all('p', attrs={'class': 'text-sm font-normal text-gray-700 text-loading-animate'}) # Posting Time Tag
    Outter_divs = soup.find_all("div", attrs={'class':'flex flex-wrap mt-3 text-sm text-gray-500 md:py-0'}) # The Outter tag contains location, Full Time and salary tags

    # Get all job URL tags first
    job_url_tags = soup.find_all('a', attrs={'data-cy':"listing-title-link"})
    # Extract hrefs into a temporary list for iteration
    temp_job_urls = [url_tag.get('href') for url_tag in job_url_tags]

    num_jobs = len(job_title)
    for i in range(num_jobs):

        #Title
        job_title_list.append(job_title[i].text)

        #company name
        company =companyNameTag[i].text
        company = company.replace('\n', '').strip()
        company_list.append(company)

        # Job Function
        job_function =jobFunctionTag[i].text
        job_function = job_function.replace('\n', '').strip()
        job_function_list.append(job_function)

        # Posting Time
        time_posted = TimeTag[i].text # time_posted
        time_posted = time_posted.replace('ago', '')
        TimeList.append(time_posted)

        #Location, Work Type, Salary
        div = Outter_divs[i]
        spans_in_div = div.find_all('span')

        location = spans_in_div[0].text.replace('\n', '').strip() if len(spans_in_div) > 0 else "N/A"
        location_list.append(location)

        workType = spans_in_div[1].text.replace('\n', '').strip() if len(spans_in_div) > 1 else "N/A"
        workType_list.append(workType)

        salary = spans_in_div[2].text.replace('\n', '').strip() if len(spans_in_div) > 2 else "N/A"
        Salary_list.append(salary)

        # job_urls - Now append the string URL from temp_job_urls
        current_job_url = temp_job_urls[i]
        job_urls_final_list.append(current_job_url)

        # Job Details - call scrapeJobDetails with the actual string URL
        current_industry, min_qual, exp_level, exp_length = scrapeJobDetails(current_job_url)

        industry_list.append(current_industry)
        Min_Qualification_list.append(min_qual)
        ExperienceLevel_list.append(exp_level)
        ExperienceLength_list.append(exp_length)


    # converting job details into a Dictionary 'Dict' - moved outside the loop
    Dict = {
        'company_name' : company_list,
        'job_title': job_title_list,
        'job_function': job_function_list,
        'industry': industry_list,
        'workType': workType_list,
        'Location':location_list,
        'Posted_Time':TimeList,
        'Min_Qualification': Min_Qualification_list,
        'ExperienceLevel': ExperienceLevel_list,
        'ExperienceLength': ExperienceLength_list,
        'Salary_range':Salary_list,
        'job_urls': job_urls_final_list
        }
    return Dict

In [ ]:
scrap_jobs()

{'company_name': ['Blossom VA Services',
  'Ibat Travels and Tour',
  'Iwule Limited',
  'Anonymous Employer',
  'Stock Stream Solutions Nig Ltd',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting',
  'Odixcity Consulting'],
 'job_title': ['TikTok Poster',
  'Graphics & Brand Manager',
  'Business Development Lead',
  'Campus Ambassador',
  'Accountant',
  'Flutter Developer',
  'Next Js Developer',
  'Vue Js Developer',
  'AI Engineer',
  'PHP Developer',
  'Full Stack Developer',
  'Front-End Developer',
  'Back-End Developer',
  'Cloud Engineer',
  'Shopify Developer',
  'Word Press Developer'],
 'job_function': ['Marketing & Communications',
  'Creative & Design',
  'Management & Business Development',
  'Marketing & Communications',
  'Accounting, Auditing & Finance',
  'Software & Data

In [ ]:
def scrap_pages(num_pages=1):
  df = pd.DataFrame()
  for count, i in enumerate(range(num_pages),1):
    if count % 10 == 0:
      print(f'Scraping page {i}')
      time.sleep(2)
    url = f'https://www.jobberman.com/jobs?_gl=1%2A1d4etjg%2A_up%2AMQ..%2A_gs%2AMQ..&gbraid=0AAAAADmiEX1l_xG5KDvNbdGx_6oOLvw1L&gclid=CjwKCAiA3fnJBhAgEiwAyqmY5bjRXQJqCH1h--u8jTguvWn1E8_GbKAW9FjKY__DimPqp4AojSNQXxoCxG4QAvD_BwE&page={i}'
    job_dict = scrap_jobs(url)
    df_temp = pd.DataFrame(job_dict)
    df = pd.concat([df, df_temp], ignore_index=True)
  return df

In [ ]:
scrap2 = scrap_pages(num_pages=10)

Scraping page 9


In [ ]:
scrap2

,company_name,job_title,job_function,Location,Posted_Time,workType,industry,Min_Qualification,ExperienceLevel,ExperienceLength,Salary_range,job_urls
0,Blossom VA Services,TikTok Poster,Marketing & Communications,Lagos,Yesterday,Full Time,"Advertising, Media & Communications",Degree,Entry level,2 years,"NGN 150,000 - 250,000",https://www.jobberman.com/listings/tiktok-post...
1,Ibat Travels and Tour,Graphics & Brand Manager,Creative & Design,Lagos,1 week,Full Time,Tourism & Travel,OND,Entry level,1 year,"NGN 150,000 - 250,000",https://www.jobberman.com/listings/graphics-br...
2,Iwule Limited,Business Development Lead,Management & Business Development,Lagos,1 week,Full Time,Manufacturing & Warehousing,HND,Mid level,3 years,"NGN 250,000 - 400,000",https://www.jobberman.com/listings/business-de...
3,Anonymous Employer,Campus Ambassador,Marketing & Communications,Rest of Nigeria (Nationwide),1 week,Part Time,IT & Telecoms,OND,Entry level,No Experience/Less than 1 year,"NGN 70,000 - 150,000",https://www.jobberman.com/listings/campus-amba...
4,Stock Stream Solutions Nig Ltd,Accountant,"Accounting, Auditing & Finance",Abuja,2 weeks,Full Time,"Retail, Fashion & FMCG",Degree,Entry level,2 years,"NGN 70,000 - 150,000",https://www.jobberman.com/listings/accountant-...
...,...,...,...,...,...,...,...,...,...,...,...,...
155,Senior Serving Saint Ltd.,Marketing and Media Officer,Marketing & Communications,Ibadan & Oyo State,3 weeks,Full Time,Tourism & Travel,Degree,Entry level,1 year,"NGN 70,000 - 150,000",https://www.jobberman.com/listings/marketing-a...
156,Yousheng Group,Customer Service Representative,Customer Service & Support,Lagos,3 weeks,Full Time,IT & Telecoms,High School (S.S.C.E),Entry level,1 year,"NGN 250,000 - 400,000",https://www.jobberman.com/listings/call-center...
157,Solar Pro Energy Limited,Marketing Designer & Social Media Executive,Marketing & Communications,Lagos,3 weeks,Full Time,Energy & Utilities,HND,Entry level,1 year,"NGN 70,000 - 150,000",https://www.jobberman.com/listings/marketing-d...
158,Eyinimofe World,Field Sales Agent,Sales,Lagos,3 weeks,Full Time,"Retail, Fashion & FMCG",OND,Mid level,2 years,Confidential,https://www.jobberman.com/listings/field-sales...


In [70]:
scrap = scrap_pages(num_pages=210)

Scraping page 9
Scraping page 19
Scraping page 29
Scraping page 39
Scraping page 49
Scraping page 59
Scraping page 69
Scraping page 79
Scraping page 89
Scraping page 99
Scraping page 109
Scraping page 119
Scraping page 129
Scraping page 139
Scraping page 149
Scraping page 159
Scraping page 169
Scraping page 179
Scraping page 189
Scraping page 199
Scraping page 209


In [73]:
scrap.head(10)

,company_name,job_title,job_function,industry,workType,Location,Posted_Time,Min_Qualification,ExperienceLevel,ExperienceLength,Salary_range,job_urls
0,Blossom VA Services,TikTok Poster,Marketing & Communications,"Advertising, Media & Communications",Full Time,Lagos,Yesterday,Degree,Entry level,2 years,"NGN 150,000 - 250,000",https://www.jobberman.com/listings/tiktok-post...
1,Ibat Travels and Tour,Graphics & Brand Manager,Creative & Design,Tourism & Travel,Full Time,Lagos,1 week,OND,Entry level,1 year,"NGN 150,000 - 250,000",https://www.jobberman.com/listings/graphics-br...
2,Iwule Limited,Business Development Lead,Management & Business Development,Manufacturing & Warehousing,Full Time,Lagos,1 week,HND,Mid level,3 years,"NGN 250,000 - 400,000",https://www.jobberman.com/listings/business-de...
3,Anonymous Employer,Campus Ambassador,Marketing & Communications,IT & Telecoms,Part Time,Rest of Nigeria (Nationwide),1 week,OND,Entry level,No Experience/Less than 1 year,"NGN 70,000 - 150,000",https://www.jobberman.com/listings/campus-amba...
4,Stock Stream Solutions Nig Ltd,Accountant,"Accounting, Auditing & Finance","Retail, Fashion & FMCG",Full Time,Abuja,2 weeks,Degree,Entry level,2 years,"NGN 70,000 - 150,000",https://www.jobberman.com/listings/accountant-...
5,Odixcity Consulting,Flutter Developer,Software & Data,Recruitment,Full Time,Remote (Work From Home),Yesterday,Degree,Entry level,2 years,"NGN 400,000 - 600,000",https://www.jobberman.com/listings/flutter-dev...
6,Odixcity Consulting,Next Js Developer,Software & Data,Recruitment,Full Time,Remote (Work From Home),Yesterday,Degree,Entry level,2 years,"NGN 400,000 - 600,000",https://www.jobberman.com/listings/next-js-dev...
7,Odixcity Consulting,Vue Js Developer,Software & Data,Recruitment,Full Time,Remote (Work From Home),Yesterday,Degree,Mid level,3 years,"NGN 400,000 - 600,000",https://www.jobberman.com/listings/vue-js-deve...
8,Odixcity Consulting,AI Engineer,Engineering & Technology,Recruitment,Full Time,Remote (Work From Home),Yesterday,Degree,Mid level,3 years,"NGN 400,000 - 600,000",https://www.jobberman.com/listings/ai-engineer...
9,Odixcity Consulting,PHP Developer,Engineering & Technology,Recruitment,Full Time,Remote (Work From Home),Yesterday,Degree,Mid level,3 years,"NGN 400,000 - 600,000",https://www.jobberman.com/listings/php-develop...


In [74]:
scrap.to_csv('job_data.csv2')